# Channels' tags network

In [1]:
import os
import sys
import nltk
import math
import shlex
import itertools
import networkx as nx
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from string import punctuation

from pandas import Series
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

In [2]:
# loading data
data_folder_channels='/media/aruiz/data/channels_clean_data/'
fileName2=os.path.join(data_folder_channels,'all_channels_cat_clean.csv')
# fileName2=os.path.join(data_folder_channels,'gta_data_clean.csv')
df=pd.read_csv(fileName2)
df=df.sort_values(by=['views'],ascending=True)
df=df.fillna("N/A")
df=df.reset_index(drop=True)
lemmatizer = WordNetLemmatizer()
stopWords = set(stopwords.words('english'))
searchKeys=["minecraft","roblox","call", "of", "duty","overwatch", "grand","theft","auto","league","legend",
            "Happy","wheel","five", "night", "at", "freddy's","agar.io", "pokemon"]
# searchKeys=["grand","theft","auto"]
top_words= ["game", "gaming", "video", "gameplay", "funny", "play", "let","cod", "mod", "tutorial"]
# quartile3=246352
quartile3=42267
#punctuation list to exclude from keywords
exclude = set(punctuation)

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Write keywords per channel in a file

In [3]:
#Method to delete numbers and punctuation signs
def remove_dig_punct(list1):
    new=[]
    for n in range (0, len(list1)):
        #removing digits
        res=''.join([i for i in list1[n] if not i.isdigit()])
        #removing punctuation
        s = ''.join(ch for ch in res if ch not in exclude)
        if (s !="") and (s not in new):
            if (len(s)>1):
                new.append(s)
    return (new)

In [4]:
dictionaries=[]
for n in range(0,len(df)):
    if (df.keywords[n]!="N/A") and (df.views[n]>quartile3):
        d1={}
        channel_keys=[]
        
        try:
            l = shlex.split(df.keywords[n])

        except:
            l = df.keywords[n].split()
            
        for kw in l:
            kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "").replace("`", "")
            single_keys=nltk.word_tokenize(kw_id)
            for key in single_keys:
                key=lemmatizer.lemmatize(key)
                if (key not in stopWords)and(len(key)>1):
                    if (key not in channel_keys) and (key not in searchKeys):
                        channel_keys.append(key)
                        
#         clean all digits and punctuation from keywords
        channel_keys_clean=remove_dig_punct(channel_keys)
                    
#         df.loc[n,"num_tags"]=len(channel_keys_clean)
        for key in channel_keys_clean: d1[key]=True
        d1["id"]=df.id[n]
        dictionaries.append(d1)

### Construct graph network with weighted edges (w = number of shared tags)

In [5]:
# create nodes with weight and title
G=nx.Graph()
for n in range(0,len(dictionaries)):
    comp1=dictionaries[n]
    channel=df.loc[df["id"]==comp1["id"],["title", "views"]]
    G.add_node(comp1["id"], name=channel.title.values[0], type='channel', weight=channel.views.values[0])               
            

In [8]:
# create edges with weight based on tag similarity
for n in range(0,len(dictionaries)):
    comp1=dictionaries[n]
    if ((n+1)<=(len(dictionaries)-1)):
        for m in range (n+1, len(dictionaries)):
            comp2=dictionaries[m]
            shared_items = len(set(comp1.items()) & set(comp2.items()))
            if (shared_items>=5):
                G.add_edge(comp1["id"],comp2["id"], weight=shared_items) 

## Merge nodes 

In [149]:
def merge_nodes(new_node, node_list, gf):
    H =gf.copy()
    gf.add_node(new_node, type='keyword') # Add the 'merged' node   

    for n1,n2,data in H.edges(data=True):
        # For all edges related to one of the nodes to merge,
        # make an edge going to or coming from the `new gene`.
        if n1 in node_list:
            g.add_edge(new_node,n2)
        elif n2 in node_list:
            g.add_edge(n1,new_node)

    for n in node_list: # remove the merged nodes
        gf.remove_node(n)

### Create network

In [13]:
H=G.copy()
a=list(nx.isolates(G))
H.remove_nodes_from(a)

In [15]:
len(H.nodes())

7724

In [21]:
nx.write_gexf(H,"similarity_tags.gexf")

KeyError: <class 'numpy.int64'>